In [ ]:
!nvidia-smi

Sun Nov 14 09:28:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip -q install iterative-stratification

In [ ]:
!pip -q install catboost

     |████████████████████████████████| 76.3 MB 38 kB/s 


In [ ]:
import os
import gc
import sys
import time
import random
import operator
import typing as tp
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd,numpy as np

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, RepeatedKFold, StratifiedKFold
from sklearn.decomposition import PCA

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/AV/AmExpert2021/input/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/AV/AmExpert2021/input/test.csv")
ss_df = pd.read_csv("/content/drive/MyDrive/AV/AmExpert2021/input/sample_submission.csv")

train_df.shape, test_df.shape, ss_df.shape

((37748, 9), (20327, 8), (20327, 2))

In [ ]:
train_df.drop(columns='Customer_ID').duplicated().sum()

2876

In [ ]:
train_df = train_df[~train_df.drop(columns='Customer_ID').duplicated()].reset_index(drop=True)
train_df.shape

(34872, 9)

In [ ]:
train_df['PHB1_len'] = train_df['Product_Holding_B1'].apply(lambda x: len(eval(x)))
test_df['PHB1_len'] = test_df['Product_Holding_B1'].apply(lambda x: len(eval(x)))

In [ ]:
PHB1_list = train_df.Product_Holding_B1.apply(eval).values.tolist()
PHB2_list = train_df.Product_Holding_B2.apply(eval).values.tolist()
tPHB1_list = test_df.Product_Holding_B1.apply(eval).values.tolist()

In [ ]:
mlb1 = MultiLabelBinarizer()
mlb2 = MultiLabelBinarizer()

PHB1_onehot = mlb1.fit_transform(PHB1_list)
PHB2_onehot = mlb2.fit_transform(PHB2_list)
tPHB1_onehot = mlb1.transform(tPHB1_list)

PHB1_onehot.shape, PHB2_onehot.shape, tPHB1_onehot.shape

((34872, 22), (34872, 20), (20327, 22))

In [ ]:
train_df[mlb1.classes_] = PHB1_onehot
test_df[mlb1.classes_] = tPHB1_onehot

In [ ]:
d = np.dot(PHB1_onehot.T, PHB2_onehot)
d.shape

(22, 20)

In [ ]:
ddf = pd.DataFrame(d, index=mlb1.classes_, columns=mlb2.classes_)

In [ ]:
#ddf = ddf.div(ddf.sum(axis=1), axis=0)

In [ ]:
ddf.head()

,P00,P1,P10,P11,P12,P13,P14,P15,P16,P17,P18,P2,P20,P3,P4,P5,P6,P7,P8,P9
P00,309,32,22,0,80,61,0,0,136,0,0,6,0,31,16,3,61,19,206,17
P1,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
P10,8,328,0,0,0,0,0,0,0,0,0,37,0,225,609,297,490,380,1048,169
P11,1161,85,119,0,0,0,0,0,0,0,0,0,0,52,135,40,73,56,196,28
P12,25,1114,1763,143,0,0,0,0,0,0,0,105,0,988,1807,747,1280,1061,3614,653


In [ ]:
# PHB1_featres = []
# for li in PHB1_list:
#     dd = ddf.loc[li].sum().values.tolist()
#     PHB1_featres.append(dd)

# dd_cols = [f'{col}_dd' for col in ddf.columns]
# train_df[dd_cols] = PHB1_featres

In [ ]:
# tPHB1_featres = []
# for li in tPHB1_list:
#     dd = ddf.loc[li].sum().values.tolist()
#     tPHB1_featres.append(dd)

# dd_cols = [f'{col}_dd' for col in ddf.columns]
# test_df[dd_cols] = tPHB1_featres

In [ ]:
train_df.shape, test_df.shape

((34872, 32), (20327, 31))

In [ ]:
cat_cols = ['Gender', 'City_Category', 'Customer_Category']

for col in cat_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])

In [ ]:
cid_list = []
target_list = []
for cid, ph_list in zip(train_df.Customer_ID.values, PHB2_list):
    for l in ph_list:
        cid_list.append(cid)
        target_list.append(l)

target_df = pd.DataFrame({
    "Customer_ID" : cid_list,
    "Target" : target_list
})

print(len(cid_list), len(target_list))

target_df.shape

49123 49123


(49123, 2)

In [ ]:
def pca_pre(tr,te, n_comp, feat_raw, feat_new):
    pca = PCA(n_components=n_comp, random_state=42)
    tr2 = pd.DataFrame(pca.fit_transform(tr[feat_raw]),columns=feat_new)
    #va2 = pd.DataFrame(pca.transform(va[feat_raw]),columns=feat_new)
    te2 = pd.DataFrame(pca.transform(te[feat_raw]),columns=feat_new)
    return(tr2,te2)

n_comp1 = 5
feat_cols = mlb1.classes_
pca_feat_g = [f'pca-{i}' for i in range(n_comp1)]

x_tr_g_pca,x_te_g_pca = pca_pre(train_df, test_df,
                                            n_comp1, feat_cols ,pca_feat_g)
train_df = pd.concat([train_df, x_tr_g_pca],axis = 1)
# x_valid = pd.concat([x_valid,x_va_g_pca],axis = 1)
test_df  = pd.concat([test_df, x_te_g_pca],axis = 1)


In [ ]:
train_df = target_df.merge(train_df, how='left')

In [ ]:
target_le = LabelEncoder()
train_df['Target'] = target_le.fit_transform(train_df['Target'])

In [ ]:
drop_cols = ['Customer_ID', 'Target', 'Product_Holding_B1', 'Product_Holding_B2']
train_cols = train_df.drop(columns=drop_cols).columns.values
train_cols

array(['Gender', 'Age', 'Vintage', 'Is_Active', 'City_Category',
       'Customer_Category', 'PHB1_len', 'P00', 'P1', 'P10', 'P11', 'P12',
       'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P2', 'P20',
       'P21', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'pca-0', 'pca-1',
       'pca-2', 'pca-3', 'pca-4'], dtype=object)

In [ ]:
target = train_df['Target'].values
X_test = test_df[train_cols].values

In [ ]:
from typing import Optional, Union, Tuple
def binary_logloss_for_lgbm(label, preds):
    """Calculate Binary Logloss"""
    # print(preds)
    # print(label)
    preds = preds.reshape(20, len(label)).T
    preds = np.argsort(preds)[:, -3:][:, ::-1]
    label = label.reshape(len(label), 1)
    #print(preds)
    #print(label)
    loss = mapk(label, preds, k=3)
    #print(loss)

    # # eval_name, eval_result, is_higher_better
    return 'my_lnloss', loss, True

In [ ]:
class ProfitMetric:
    def is_max_optimal(self):
        return True # greater is better

    def evaluate(self, y_pred, y_true, weight):
        
        y_pred = np.array(y_pred).T
        y_pred = np.argsort(y_pred)[:, -3:][:, ::-1]

        y_true = np.array(y_true).reshape(len(y_true), 1)
        score = mapk(y_true, y_pred, k=3)

        return score, 1

    def get_final_error(self, error, weight):
        return error

In [ ]:
#skf = MultilabelStratifiedKFold(n_splits= 5, shuffle=True, random_state=42)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

test_predictions = []

for idx, (train_idx, valid_idx) in enumerate(skf.split(train_df, target)):
    
    X_train = train_df.iloc[train_idx][train_cols].values
    y_train = target[train_idx]

    X_valid = train_df.iloc[valid_idx][train_cols].values
    y_valid = target[valid_idx]

    print("Trian :", X_train.shape, y_train.shape)
    print("Valid :", X_valid.shape, y_valid.shape)

    params = {'iterations':10000, 
              #'depth': 8, 
              #'boosting_type': 'Ordered', #'Ordered', #'Plain',
              'loss_function': 'MultiClass', 
              'eval_metric': 'MultiClass',#ProfitMetric() ,#'MultiClass',
              #'learning_rate': 5e-2, 'leaf_estimation_method': 'Gradient', #'Newton', 'Exact'
              #'l2_leaf_reg': 1.0, 'random_strength': 1.0, 'bagging_temperature': 1.0, 'has_time': False,
              #'grow_policy': 'SymmetricTree', #'Depthwise', 'Lossguide'
              #'min_data_in_leaf': 1, 'max_leaves': 31,
              'random_seed': 43,
             #'one_hot_max_size': len(cat_features),
                'classes_count' : 20,
                'task_type' : "GPU",
              }
    
    #params = {'boosting_type': 'gbdt', 'num_class':20}


    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds=100, verbose=50)
    # pred_val_y = model.predict_proba(val_X)

    # pred_val_y = model.predict_proba(val_X)
    predictions = model.predict_proba(X_test)

    test_predictions.append(predictions)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Trian : (39298, 34) (39298,)
Valid : (9825, 34) (9825,)
Learning rate set to 0.055701
0:	learn: 2.7956603	test: 2.7940559	best: 2.7940559 (0)	total: 13.3ms	remaining: 2m 13s
50:	learn: 1.6796710	test: 1.6824316	best: 1.6824316 (50)	total: 580ms	remaining: 1m 53s
100:	learn: 1.5970166	test: 1.6087856	best: 1.6087856 (100)	total: 1.13s	remaining: 1m 50s
150:	learn: 1.5690203	test: 1.5902605	best: 1.5902605 (150)	total: 1.71s	remaining: 1m 51s
200:	learn: 1.5499811	test: 1.5811106	best: 1.5811106 (200)	total: 2.25s	remaining: 1m 49s
250:	learn: 1.5357833	test: 1.5778172	best: 1.5778172 (250)	total: 2.81s	remaining: 1m 48s
300:	learn: 1.5238096	test: 1.5771291	best: 1.5767161 (278)	total: 3.35s	remaining: 1m 47s
350:	learn: 1.5139266	test: 1.5778003	best: 1.5767161 (278)	total: 3.92s	remaining: 1m 47s
bestTest = 1.576716066
bestIteration = 278
Shrink model to first 279 iterations.
Trian : (39298, 34) (39298,)
Valid : (9825, 34) (9825,)
Learning rate set to 0.055701
0:	learn: 2.7959854	test

In [ ]:
final_predictions_mean = np.array(test_predictions).mean(axis=0)
final_predictions_mean.shape

(20327, 20)

In [ ]:
np.save("/content/drive/MyDrive/AV/AmExpert2021/input/CAT_D_M_pre.npy", final_predictions_mean)

In [ ]:
final_predictions = np.argsort(final_predictions_mean)[:, -3:][:, ::-1]

In [ ]:
final_test = []
for i in final_predictions:
    final_test.append([target_le.classes_[int(ii)] for ii in i])

In [ ]:
ss_df['Product_Holding_B2'] = final_test
ss_df.to_csv("/content/drive/MyDrive/AV/AmExpert2021/input/CAT_D_M.csv",index=False)